In [4]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기

In [9]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [13]:
!mkdir preview

In [14]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview',
                                 save_prefix='tri', save_format='png'):
    i += 1
    if i > 30:
        break

In [15]:
!ls preview

tri_0_1270.png	tri_0_3825.png	tri_0_5881.png	tri_0_8211.png	tri_0_9797.png
tri_0_1302.png	tri_0_403.png	tri_0_6416.png	tri_0_8288.png	tri_0_9854.png
tri_0_2407.png	tri_0_4949.png	tri_0_6427.png	tri_0_8923.png	tri_0_9972.png
tri_0_2997.png	tri_0_5158.png	tri_0_7039.png	tri_0_9089.png
tri_0_3418.png	tri_0_519.png	tri_0_7259.png	tri_0_928.png
tri_0_3503.png	tri_0_5672.png	tri_0_7282.png	tri_0_9495.png
tri_0_3759.png	tri_0_5798.png	tri_0_7897.png	tri_0_9575.png


In [16]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 15 images belonging to 3 classes.


In [19]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15 * 100,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
1500/1500 [==============================] - 7s 191us/step - loss: 1.2986 - accuracy: 0.3335 - val_loss: 1.1003 - val_accuracy: 0.3333


In [22]:
print("-- Evaluate --")

scores = model.evaluate_generator(
            test_generator, 
            steps = 5)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 33.33%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [24]:
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

-- Predict --
[[0.385 0.295 0.320]
 [0.353 0.295 0.352]
 [0.355 0.297 0.347]
 [0.359 0.291 0.350]
 [0.360 0.291 0.349]
 [0.391 0.287 0.322]
 [0.351 0.289 0.360]
 [0.371 0.299 0.331]
 [0.381 0.291 0.327]
 [0.349 0.300 0.351]
 [0.374 0.294 0.332]
 [0.376 0.295 0.329]
 [0.351 0.298 0.351]
 [0.368 0.292 0.340]
 [0.360 0.296 0.344]]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
